## **Model Evaluation**

## Importing stuff and loading the model for evaluation

In [ ]:
from fastbook import *
from glob import glob
from pathlib import Path
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, roc_auc_score, classification_report, confusion_matrix 
from tqdm.notebook import tqdm

In [ ]:
learn_inf = load_learner('dbc_resnet50_fastai_bigv3.5-cleaned.pkl')

In [ ]:
%cd eval

In [ ]:
!pwd
!find . -type f ! -name '*.jpg' -delete

## Running The Evaluation

In [ ]:
def get_topk(model_output, k=3, out_type="dict"):
    predictions = sorted(zip(model_output[2].tolist(), learn_inf.dls.vocab), reverse=True)
    top_k = predictions[:k]
    if out_type == "dict":
        return top_k
    elif out_type == "list":
        return list(dict(top_k).values())
    else:
        raise Exception("Invalid Output Type")

In [ ]:
from tqdm.notebook import tqdm

%cd /home/studio-lab-user/sagemaker/eval

truthlist = []
predictionlist = []

#for folder in tqdm(sorted(os.listdir('.'))):
print(f"Categories to evaluate: {len(learn_inf.dls.vocab)}")
for folder in tqdm(sorted(learn_inf.dls.vocab)):
  os.chdir(folder)
  tqdm.write(f"Evaluating accuracy for folder {folder}")
  truth = folder.lower()
  for file in tqdm(sorted(os.listdir('.'))):
    if file.endswith(".jpg"):
        top3 = get_topk(learn_inf.predict(file), out_type="list")
        truthlist.append(folder)
        predictionlist.append(top3)
    else:
      print("Skipping Checkpoint File - Not an image")
  os.chdir('..')

%cd /home/studio-lab-user/sagemaker

In [ ]:
tbackup = truthlist.copy()
pbackup = predictionlist.copy()

In [ ]:
acceptedlist = []
for i in range(len(truthlist)):
  #print(truthlist[i])
  #print(predictionlist[i])
  if truthlist[i] in predictionlist[i]:
    #print("correct")
    acceptedlist.append(truthlist[i])
  else:
    #print("incorrect")
    acceptedlist.append(predictionlist[i][0])
#print(classification_report(truthlist, acceptedlist))
print("Done.")

## Saving the evaluation output to JSON file
 - Allows more processing to be done to the data later if required without needing to re-run the model

In [ ]:
import json

def dump_eval_predictions(truths, predictions, accepted, version):
    filename = f"cat-v{version}-eval.json"
    data = {"truthlist": truths, "predictionlist": predictions, "acceptedlist": accepted}
    with open(filename, "w") as file:
        json.dump(data, file, indent=2)
        print(f"Successfully Dumped Evaluation Data to file {filename}\nItem Count -> T:{len(data['truthlist'])} P:{len(predictionlist)} A:{len(acceptedlist)}")
        
        
def load_eval_predictions(version):
    filename = f"cat-v{version}-eval.json"
    with open(filename, "r") as file:
        data = json.load(file)
        print(f"Successfully Loaded Evaluation Data from file {filename}\nItem Count -> T:{len(data['truthlist'])} P:{len(data['predictionlist'])} A:{len(data['acceptedlist'])}")
        print(f"Usage: truthlist, predictionlist, acceptedlist = load_eval_predictions(#)\n")
        return [data['truthlist'], data['predictionlist'], data['acceptedlist']]
        

In [ ]:
dump_eval_predictions(truthlist, predictionlist, acceptedlist, "3.5-new")

## Loading The Data

In [ ]:
%cd ~/sagemaker
truthlist, predictionlist, acceptedlist = load_eval_predictions("3.5-new")

## Data Analysis
 - Classification Report
 - Confusion Matrix

In [ ]:
print(classification_report(truthlist, acceptedlist))